In [1]:
import requests
import json
import logging
from src.tournament_result_sources.edh_top_16_source import EdhTop16Source
from src.deck_sources.deck_source_factory import DeckSourceFactory
from src.deck.deck import Deck

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
source = EdhTop16Source()
deck_list_source_factory = DeckSourceFactory()

In [2]:
df = source.get_all_decks()
len(df)

7042

In [3]:
trimmed_df = df[["winRate", "wins", "standing", "decklist"]]
trimmed_df["deck"] = trimmed_df["decklist"].apply(
    deck_list_source_factory.get_deck
)

/tmp/ipykernel_19601/1322275988.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_df["deck"] = trimmed_df["decklist"].apply(


In [4]:
trimmed_df = trimmed_df.sort_values("winRate", ascending=False).reset_index(drop=True)
trimmed_df = trimmed_df.dropna()
len(trimmed_df)

6377

In [12]:
def get_commanders_for_card(card_name: str) -> list[str]:
    specific_df = trimmed_df[trimmed_df.apply(lambda row: row['deck'].contains(card_name) , axis=1)].reset_index(drop=True)
    specific_list = list(specific_df["deck"])
    commanders = list(map(lambda deck: deck.commanders, specific_list))
    commanders = list(map(lambda commanders: list(map(lambda commander: commander["name"], commanders)), commanders))
    return commanders

In [5]:
decklist_data = trimmed_df.copy()
list_of_all_cards: list[list[str]] = []
list_of_commanders: list[list[str]] = []
for row in decklist_data.iterrows():
    list_of_all_cards.append(sorted(list({card.name for card in row[1]['deck'].maindeck})))
    list_of_commanders.append(sorted(list({card.name for card in row[1]['deck'].commanders})))
decklist_data["cards"] = list_of_all_cards
decklist_data["commanders"] = list_of_commanders
card_name_df = decklist_data.copy()[["winRate", "wins", "standing", "cards", "commanders"]]
commanders_and_cards = card_name_df.explode("cards")[["cards", "commanders"]].groupby("cards", as_index=False).sum()
commanders_and_cards["commanders"] = commanders_and_cards["commanders"].apply(set).apply(list)
count_cards = card_name_df.explode("cards")[["cards", "commanders"]].groupby("cards", as_index=False).count().rename(columns={"commanders": "count"})
card_stats = card_name_df[["winRate", "wins", "standing", "cards"]].explode("cards").groupby("cards", as_index=False).mean().sort_values("winRate", ascending=False).reset_index(drop=True)
commanders_and_cards

,cards,commanders
0,"""Name Sticker"" Goblin","[Godo, Bandit Warlord]"
1,Aarakocra Sneak,[Sefris of the Hidden Ways]
2,Abandon Reason,[Anje Falkenrath]
3,"Abdel Adrian, Gorion's Ward","[Preston, the Vanisher, Tivit, Seller of Secre..."
4,Abeyance,"[Ao, the Dawn Sky, Hinata, Dawn-Crowned, Kaali..."
...,...,...
5749,"Zurzoth, Chaos Rider","[Ob Nixilis, Captive Kingpin, Armix, Filigree ..."
5750,_____ Goblin,"[Ob Nixilis, Captive Kingpin, Dihada, Binder o..."
5751,"Éomer, King of Rohan","[Winota, Joiner of Forces]"
5752,"Éomer, Marshal of Rohan","[Winota, Joiner of Forces]"


In [6]:
card_data = commanders_and_cards.merge(card_stats, on="cards").merge(count_cards, on="cards").sort_values("winRate", ascending=False).reset_index(drop=True)
card_data_as_list = []
for row in card_data[card_data["count"] > 100][["cards", "winRate", "count"]].iterrows():
    card_data_as_list.append(dict(row[1]))

In [7]:
with open("card_data.json", "w") as f:
    json.dump(card_data_as_list, f, indent=4)